# Import the dependencies

In [1]:
import os
from NSE_tool.Main import Manager as NSE
from pprint import pprint
import sys 
print(sys.version)

'''
###########################################################
The manager is the instance linking the following parts:
- I/O manager for input output
- The processing of data 
- The fitting of data (maybe merge processing and fitting)
- The graphical manopulation of reduced and fitted data
###########################################################
'''
tool = NSE()



ModuleNotFoundError: No module named 'Log'

# Process the elastic data

## First set up the environnement

In [2]:
'''
###########################################################
INITIALISE THE ENVIRONEMENT:

Each dataset is initialised in a new environement. We
therefore initialise a new environement through the manager
run command with the keyword 'new environement'. The
title and selected type are optional and will fall back to 
'No_title' and 'MIEZE' if not provided.

This command is equivalent to performeing:
tool.run('new data', title = 'elastic', select = 'MIEZE')
tool.run('new mask', title = 'elastic', select = 'MIEZE')
tool.run('new fit',  title = 'elastic', select = 'MIEZE')

###########################################################
'''
elastic_env = tool.new_environment(title = 'elastic', select = 'SANS')


## Load the data

In [3]:
'''
###########################################################
LOAD THE DATA:

at a further point in time different routines will be 
provided to load data. The current way of proceedign is to 
load a file containing all relevant information and let the
routine parse the content. 
###########################################################
'''
elastic_env.io.load_SANS_PAD('SkX_down.txt')
elastic_env.process.extract_from_metadata(axis = 'Temperature', key = 'T_B')
print(elastic_env.current_data)

=========== This is the Input ============

['Data', "['110_1']", '0012', 'range(2194, 2298)', '(128x128)']
['Data', "['100_1']", '0012', 'range(2681, 2725)', '(128x128)']
['Data', "['100_2']", '0012', 'range(2890, 2956)', '(128x128)']
['Data', "['BG']", '0012', 'range(1609, 1611)', '(128x128)']

##########################################################
################## DATA STRUCTURE ########################
##########################################################
The datastructure currently consists of:
- Number of elements: 108
- Number of dimensions: 2
- The axes names are: ['Type', 'Temperature']
- The axes units are: ['-', 'K']
- The axes lengths are: [4, 106]
- The base structure is: np
- The base structure dim is: (128, 128)
- The metadata addition is set to: True
----------------------------------------------------------
Creation date: Fri Sep 21 08:32:30 2018
Source format: PAD files
Measurement type: SANS
##########################################################




## Create a Mask anprocess intensities

In [4]:
'''
###########################################################
SET THE MASK:

A large variety of masks are available to be edited. Using
the 'set mask template' keyword it is possible to set a 
predefined mask. Further implementations will allow the 
change of the default mask parameters and at some point the 
loading of custome masks from file.
###########################################################
'''
elastic_env.mask.select_template(key = 'SkX_peak_SkXCon')
elastic_env.process.calculate_intensity()
elastic_result_1 = elastic_env.get_result(name = 'Intensity')

elastic_env.mask.select_template(key = 'SkX_peak_Sixfold')
elastic_env.process.calculate_intensity()
elastic_result_2 = elastic_env.get_result(name = 'Intensity')


# Vertical data

## Create environement and load data

In [5]:
vertical_env = tool.new_environment(title = 'vertical MIEZE', select = 'MIEZE')
vertical_env.io.load_MIEZE_TOF('LoadSkXCon.txt')
print(vertical_env.current_data)

=========== This is the Input ============

['Data', '[28.40,0]', '0012', 'range(2727, 2736)', '(8x16x128x128)']
['Data', '[62.00,0]', '0012', '[2736]+[i for i in range(2739, 2747)]', '(8x16x128x128)']
['Data', '[28.60,0]', '0012', 'range(2747, 2756)', '(8x16x128x128)']
['Data', '[28.80,0]', '0012', 'range(2813, 2822)', '(8x16x128x128)']
['Data', '[28.95,0]', '0012', 'range(2774, 2783)', '(8x16x128x128)']
['Data', '[29.05,0]', '0012', 'range(2804, 2813)', '(8x16x128x128)']
['Data', '[29.15,0]', '0012', 'range(2765, 2774)', '(8x16x128x128)']
['Data', '[29.25,0]', '0012', 'range(2858, 2867)', '(8x16x128x128)']
['Data', '[29.35,0]', '0012', 'range(2756, 2765)', '(8x16x128x128)']
['Data', '[29.45,0]', '0012', 'range(2849, 2858)', '(8x16x128x128)']
['Data', '[29.60,0]', '0012', 'range(2792, 2801)', '(8x16x128x128)']
['Data', '[29.75,0]', '0012', 'range(2783, 2792)', '(8x16x128x128)']
['Data', '[30.50,0]', '0012', 'range(2840, 2849)', '(8x16x128x128)']

######################################

## Set the mask

In [6]:
vertical_env.mask.select_template(key = 'Pre_SkX_peak_SkXCon')
print(vertical_env.mask)


##########################################################
################## MASK STRUCTURE ########################
##########################################################
The mask has been set as follows: 
- Selected mask template: Pre_SkX_peak_SkXCon
- Parameters:
          (28, 34)
          65
          85
          (80, 104)
          10
          11
----------------------------------------------------------
##########################################################




## Set the fitting parameters

In [7]:
############################################
#initialize them
foils_in_echo = []
for i in range(5):
    foils_in_echo.append([1,1,1,1,1,1])
foils_in_echo.append([1,1,0,1,1,1])
foils_in_echo.append([1,1,0,1,1,1])
foils_in_echo.append([0,0,0,0,1,0])
foils_in_echo.append([0,0,0,0,1,0])

#set the values to be processed as data
Select = [28.40, 28.60, 28.80, 28.95, 29.05, 29.15, 29.25, 29.35, 29.45, 29.60, 29.75]

#set the reference value
Reference = 28.6

#set the background
Background = None

vertical_env.fit.set_parameter( name = 'Select',        value = Select       )
vertical_env.fit.set_parameter( name = 'Reference',     value = Reference    )
vertical_env.fit.set_parameter( name = 'Background',    value = Background   )
vertical_env.fit.set_parameter( name = 'foils_in_echo', value = foils_in_echo)


## Process throught the fitting steps

In [8]:
vertical_env.process.calculate_echo()
vertical_env.process.remove_foils()
vertical_env.process.calculate_shift()
vertical_env.mask.select_template(key = 'SkX_peak_SkXCon')

Processing shift for 28.4 K
Processing shift for 28.6 K
Processing shift for 28.95 K
Processing shift for 29.15 K
Processing shift for 29.25 K
Processing shift for 29.35 K
Processing shift for 29.45 K
Processing shift for 29.6 K
Processing shift for 29.75 K
Processing shift for 30.5 K
Processing shift for 28.8 K
Processing shift for 29.05 K
Processing shift for 62.0 K


In [ ]:

print(vertical_env.mask)
tool.launch_panel(vertical_env)




##########################################################
################## MASK STRUCTURE ########################
##########################################################
The mask has been set as follows: 
- Selected mask template: SkX_peak_SkXCon
- Parameters:
          (28, 34)
          85
          65
          (82, 100)
----------------------------------------------------------
##########################################################




interactive(children=(Dropdown(description='para', options=(28.4, 28.6, 28.95, 29.15, 29.25, 29.35, 29.45, 29.…

In [ ]:
vertical_env.process.calculate_ref_contrast()
vertical_env.process.calculate_contrast()
vert_result = vertical_env.get_result(name = 'Contrast fit')


In [ ]:
#print(vert_result)
vert_result.log.dump_to_file('lol.txt')
vert_result.log.dump_to_console()

# Parallel Data

In [ ]:
parallel_env = tool.new_environment(title = 'parallel MIEZE', select = 'MIEZE')
parallel_env.io.load_MIEZE_TOF('SkXSixfold.txt')
print(parallel_env.current_data)


In [ ]:
parallel_env.mask.select_template(key = 'Pre_SkX_peak_Sixfold')
parallel_env.mask.add_command(command_str = 'mask.real[abs(mask.real) > 8] = 0')
print(parallel_env.mask)

In [ ]:
parallel_env.process.calculate_echo()
parallel_env.process.remove_foils()
parallel_env.process.calculate_shift()

## Set fit parameters

In [ ]:
############################################
#initialize them
foils_in_echo = []
for i in range(4):
    foils_in_echo.append([1,1,1,1,1,1])
foils_in_echo.append([1,1,0,1,1,1])
foils_in_echo.append([1,1,0,1,1,1])
foils_in_echo.append([0,0,0,0,1,0])
foils_in_echo.append([0,0,0,0,1,0])


#set the values to be processed as data
Select = [ 28.6, 29.0, 29.1, 29.2, 29.4, 29.6, 29.8, 30.0]

#set the reference value
Reference = 28.6

#set the background
Background = 68.0

parallel_env.fit.set_parameter( name = 'Select',        value = Select       )
parallel_env.fit.set_parameter( name = 'Reference',     value = Reference    )
parallel_env.fit.set_parameter( name = 'Background',    value = Background   )
parallel_env.fit.set_parameter( name = 'foils_in_echo', value = foils_in_echo)

## Process to fit the data

In [ ]:
parallel_env.mask.select_template(key = 'SkX_peak_Sixfold')
print(parallel_env.mask)


parallel_env.process.calculate_ref_contrast()

#coorect values
parallel_env.results.set_result( 
         name = 'Reference contrast calculation', 
         position = ['Contrast_ref',0.36585973199337996], 
         value = 0.73)

parallel_env.results.set_result(
         name = 'Reference contrast calculation', 
         position = ['Contrast_ref_error',0.36585973199337996], 
         value = 0.0035)

parallel_env.process.calculate_contrast()
parallel_result = parallel_env.get_result(name = 'Contrast fit')


# Plot perp data

In [ ]:
############################################
#see what hapens
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import scipy.constants as co
from matplotlib.colors import Colormap, LogNorm
from matplotlib.cm import get_cmap
import pandas as pd
%matplotlib inline
fig = plt.figure(figsize=(16,5))
ax = fig.add_subplot(1,2,1)

Ts = []
Gammas = []
Gammaerrs = []

index = 0
def get_color(n, number_of_colors, cmapstr):

    cmap = get_cmap(cmapstr)

    return cmap(float(n)/number_of_colors)

for T in vert_result['Select']:
    
    x           = vert_result['Parameters'][T]['x']
    y           = vert_result['Parameters'][T]['y']
    y_error     = vert_result['Parameters'][T]['y_error']

    c = get_color(index,11,'tab10')
    ax.errorbar(
        x, 
        y+1.2-index*0.2,
        y_error, 
        fmt='o', 
        color=c)
        #label='$T=%.2f\,K$' %SkXCon[T].T_B)
    x = np.linspace(0.01,3,1000)

    ax.plot(x, vert_result['Curve'][T]+1.2-index*0.2, color=c)
    
    index += 1


ax.set_xscale('log')
ax.set_xlim(0.01,3)
ax.legend(bbox_to_anchor=(-0.1,0.9))
ax.text(2e-2, 0, r'$B \perp n$, $B=160\,mT$', fontsize=14)


ax1 = fig.add_subplot(1,2,2)
c = get_color(0,10,'tab10')
ax1.errorbar(
    vert_result['Select'],
    [vert_result['Gamma'][T] for T in vert_result['Select']],
    [vert_result['Gamma_error'][T] for T in vert_result['Select']],
    fmt='o', color=c, label='Franz\' quasielastic')

ax1.set_ylabel(r'$\Gamma\ (\mu eV)$')
ax1.set_xlabel(r'$T\ (K)$')
ax1.set_xlim(27,31)
ax1.legend()

ax2 = ax1.twinx()
c = get_color(1, 10, 'tab10')
ax2.errorbar(
    elastic_result_1['Axis']['100_1'],
    elastic_result_1['Intensity']['100_1'],
    yerr=elastic_result_1['Intensity_error']['100_1'],
    fmt='o', color=c, label='Franz\' elastic')


#load Jonas' data
Jdata = pd.read_csv('data/Kindervater_Bvertn_quasielastic', skiprows=6, names=['T1', 'Gamma'])
c = get_color(0, 10, 'tab10')
ax1.errorbar(Jdata['T1']+0.2, Jdata['Gamma'], fmt='s', mec=c, mew=2, label='Jonas\' elastic', color='None')

Jdata = pd.read_csv('data/Kindervater_Bvertn_intensity', skiprows=6, names=['T2', 'Int'])
c = get_color(1, 10, 'tab10')
ax2.errorbar(Jdata['T2']+0.2, Jdata['Int']/(22.*3), fmt='s', mec=c, mew=2, label='Jonas\' quasielastic', color='None')
plt.savefig('B_vert_n.pdf')

# Plot parallel data

In [ ]:
############################################
#see what hapens
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import scipy.constants as co
from matplotlib.colors import Colormap, LogNorm
from matplotlib.cm import get_cmap
import pandas as pd
%matplotlib inline
fig = plt.figure(figsize=(16,5))
ax = fig.add_subplot(1,2,1)

Ts = []
Gammas = []
Gammaerrs = []

index = 0

def get_color(n, number_of_colors, cmapstr):

    cmap = get_cmap(cmapstr)

    return cmap(float(n)/number_of_colors)

for T in parallel_result['Select']:
    
    x           = parallel_result['Parameters'][T]['x']
    y           = parallel_result['Parameters'][T]['y']
    y_error     = parallel_result['Parameters'][T]['y_error']

    c = get_color(index,10,'tab10')
    ax.errorbar(
        x, 
        y+1.2-index*0.2,
        y_error, 
        fmt='o', 
        color=c,
        label='$T=%.2f\,K$' %T)
    x = np.linspace(0.01,3,1000)

    ax.plot(x, parallel_result['Curve'][T]+1.2-index*0.2, color=c)
    
    index += 1


ax.set_xscale('log')
ax.set_xlim(0.01,3)
ax.legend(bbox_to_anchor=(-0.1,0.9))
ax.text(2e-2, 0, r'$B \perp n$, $B=160\,mT$', fontsize=14)


ax1 = fig.add_subplot(1,2,2)
c = get_color(0,10,'tab10')
ax1.errorbar(
    parallel_result['Select'],
    [parallel_result['Gamma'][T] for T in parallel_result['Select']],
    [parallel_result['Gamma_error'][T] for T in parallel_result['Select']],
    fmt='o', color=c, label='Franz\' quasielastic')

ax1.set_ylabel(r'$\Gamma\ (\mu eV)$')
ax1.set_xlabel(r'$T\ (K)$')
ax1.set_xlim(27,31)
ax1.legend()

ax2 = ax1.twinx()
c = get_color(1, 10, 'tab10')
ax2.errorbar(
    elastic_result_2['Axis']['110_1'], 
    elastic_result_2['Intensity']['110_1'], 
    yerr=elastic_result_2['Intensity_error']['110_1'], 
    fmt='o', color=c, label='Franz\' elastic')

#load Jonas' data
Jdata = pd.read_csv('data/Kindervater_Bparan_quasielastic', skiprows=6, names=['T1', 'Gamma'])
c = get_color(0, 10, 'tab10')
ax1.errorbar(Jdata['T1']+0.4, Jdata['Gamma'], fmt='s', mec=c, mew=2, label='Jonas\' elastic', color='None')

Jdata = pd.read_csv('data/Kindervater_Bparan_intensity', skiprows=6, names=['T2', 'Int'])
c = get_color(1, 10, 'tab10')
ax2.errorbar(Jdata['T2']+0.4, Jdata['Int']/(44*3), fmt='s', mec=c, mew=2, label='Jonas\' quasielastic', color='None')
plt.savefig('B_parallel_n.pdf')